# XOX PnF charts
## Development Notes

Two methods for populating charts:

- **High and Low** price: whenever H and L are available
- **Last/Close** price: for illiquid assets (or indices). Also for intraday trading using realtime data

Scale types:

- Constant size boxes
- Variable size boxes

In [1]:
import numpy as np
import pandas as pd

import sys
print(f"Python version: {sys.version}")
print(f"pandas version: {pd.__version__}")
print(f"numpy version: {np.__version__}")

Python version: 3.9.12 (main, Apr  5 2022, 01:53:17) 
[Clang 12.0.0 ]
pandas version: 1.4.2
numpy version: 1.22.3


In [2]:
# Reading a sample data file

data = pd.read_csv("data/EXP1.csv")
data.head()

,Date,Open,High,Low,Close,Volume
0,2001-01-01,37.25,38.25,35.75,37.50,1234000
1,2001-01-02,39.50,40.12,39.25,40.00,1567000
2,2001-01-03,37.50,38.50,37.00,37.25,1456000
3,2001-01-04,37.00,37.50,36.50,37.00,1789000
4,2001-01-05,37.00,40.25,37.00,39.00,2345000


In [ ]:
data.describe()

In [ ]:
data.info()

In [3]:
# Using only H, L and C data
price_data = data[['High','Low','Close']]

In [4]:
price_data.head()

,High,Low,Close
0,38.25,35.75,37.50
1,40.12,39.25,40.00
2,38.50,37.00,37.25
3,37.50,36.50,37.00
4,40.25,37.00,39.00


## PnF functions

- init_pnf() initializes the first column
- update_pnf() deals with the rest of the chart
- chart_text() generates a string with a text version of the chart

In [5]:
import pnfplot

In [ ]:
# TO DO
def generate_scale(start, end, box_size=1, method='standard'):
    scale = np.arange(start=start, stop=high+box_size, step=box_size)
    
    return scale

## Testing our functions

In [66]:
# testing init_pnf() on the 1st line of data:

box_size = 1
reversal_size = 3
# scale = pnfplot.generate_scale(start=32, end=44, box_size = box_size)
scale = np.arange(32,42, 1)

high = 38.25
low = 35.75
close = 37.5

status, box_range = pnfplot.init_pnf(scale, high, low, close, reversal_size)
print(status, box_range)

1 [36 37 38]


In [67]:
# testing init_pnf() on subsequent lines of data (when status==0)
# define: box_range array, status


high = 39.25
low = 35.01
close = 38

pnfplot.init_pnf(scale, high, low, close, reversal_size, box_range)

(1, array([36, 37, 38, 39]))

### Tests for update_pnf() go here

In [68]:
# test 1
trend_status = np.array([1, 0, 0])
box_range = np.array([36, 37, 38])

reversal_size = 3

high = 40.12
low = 39.25

start = 1
status = trend_status[0]
box_h = box_range.max()
box_l = box_range.min()

# print(scale, high, low, status, reversal_size, box_l, box_h)
pnfplot.update_pnf(scale, high, low, status, reversal_size, box_l, box_h)

(1, array([36, 37, 38, 39, 40]))

In [69]:
# test 2
trend_status = np.array([1, 1, 0])
box_range = np.array([36, 37, 38, 39, 40])

reversal_size = 3

high = 38.5
low = 37

status = trend_status[1]
box_h = box_range.max()
box_l = box_range.min()

# print(scale, high, low, status, reversal_size, box_l, box_h)
pnfplot.update_pnf(scale, high, low, status, reversal_size, box_l, box_h)

(-1, array([37, 38, 39]))

In [70]:
# test 3
trend_status = np.array([1, 1, -1])
box_range = np.array([37, 38, 39])

reversal_size = 3

high = 40.25
low = 37

start = 1
status = trend_status[2]
box_h = box_range.max()
box_l = box_range.min()

# print(scale, high, low, status, reversal_size, box_l, box_h)
pnfplot.update_pnf(scale, high, low, status, reversal_size, box_l, box_h)

(1, array([38, 39, 40]))

## Processing the data using the functions

In [71]:
price_data.head()

,High,Low,Close
0,38.25,35.75,37.50
1,40.12,39.25,40.00
2,38.50,37.00,37.25
3,37.50,36.50,37.00
4,40.25,37.00,39.00


In [84]:
# initialise status and box arrays:
trend_status = np.zeros(len(price_data))
box_low = np.zeros(len(price_data))
box_high = np.zeros(len(price_data))

trend_data = pd.DataFrame({
    'trend_status' : trend_status,
    'range_low': box_low,
    'range_high': box_high
})

pnf_data = pd.concat([price_data, trend_data], axis=1)
pnf_data

,High,Low,Close,trend_status,range_low,range_high
0,38.25,35.75,37.50,0.0,0.0,0.0
1,40.12,39.25,40.00,0.0,0.0,0.0
2,38.50,37.00,37.25,0.0,0.0,0.0
3,37.50,36.50,37.00,0.0,0.0,0.0
4,40.25,37.00,39.00,0.0,0.0,0.0


In [85]:
# initialise status and box arrays:
# trend_status = np.zeros(len(price_data))
# box_low = np.zeros(len(price_data))
# box_high = np.zeros(len(price_data))

box_size = 10
reversal_size = 3
# scale = generate_scale(start=np.floor(price_data['Low'].min()), end=np.ceil(price_data['High'].max()), box_size=box_size)
scale = np.arange(34,42, 1)

print(f'Trend status: {trend_status}\nBox Low: {box_low}\nBox High: {box_high}\nScale: {scale}')

Trend status: [0. 0. 0. 0. 0.]
Box Low: [0. 0. 0. 0. 0.]
Box High: [0. 0. 0. 0. 0.]
Scale: [34 35 36 37 38 39 40 41]


In [ ]:
# Initialise the chart until a status (+/-)1 is reached
box_range = []
for index, row in enumerate(price_data.iterrows()):
    high = row[1]['High']
    low = row[1]['Low']
    close = row[1]['Close']
    status, box_range = pnfplot.init_pnf(scale, high, low, close, reversal_size, box_range)
    trend_status[index] = status
    box_low[index] = box_range.min()
    box_high[index] = box_range.max()
    if status != 0:
        break

# status can still be 0! create an example for testing
print(f'Index: {index}\nTrend status: {trend_status}\nBox Low: {box_low}\nBox High: {box_high}\nRange: {scale[np.logical_and(scale>=box_low[index], scale<=box_high[index])]}')

In [89]:

# Alt version - Repeat initialize!
# Initialise the chart until a status (+/-)1 is reached

box_range = []
# row_count = 0
for row in pnf_data.iterrows():
    high = row[1]['High']
    low = row[1]['Low']
    close = row[1]['Close']
    status, box_range = pnfplot.init_pnf(scale, high, low, close, reversal_size, box_range)
    row[1]['trend_status'] = status
    row[1]['range_low'] = box_range.min()
    row[1]['range_high'] = box_range.max()
#     row_count += 1
    if status != 0:
        status_found = row[0]
        break

print(row[0], row_count)
pnf_data
# status can still be 0! create an example for testing

0 1


,High,Low,Close,trend_status,range_low,range_high
0,38.25,35.75,37.50,1.0,36.0,38.0
1,40.12,39.25,40.00,0.0,0.0,0.0
2,38.50,37.00,37.25,0.0,0.0,0.0
3,37.50,36.50,37.00,0.0,0.0,0.0
4,40.25,37.00,39.00,0.0,0.0,0.0


In [83]:
# Check if there are more lines of data to process
print(status_found + 1 < len(price_data)-1)

True


In [ ]:
# Next, we need to process the prices after index:
start = index + 1
price_data.loc[start:].head()

In [ ]:
# Process the remaining lines in price_data:
for index, row in enumerate(price_data.loc[start:].iterrows()):
    high = row[1]['High']
    low = row[1]['Low']
    status = trend_status[index + start - 1]
    box_l = box_low[index + start - 1]
    box_h = box_high[index + start - 1]
    status, box_range = update_pnf(scale, high, low, status, reversal_size, box_l, box_h)
    trend_status[index+start] = status
    box_low[index+start] = box_range.min()
    box_high[index+start] = box_range.max()
    print(f'Day: {index+start+1}, Trend status: {status}, High and Low: ', high, low, box_low, box_high)

# status can still be 0! create an example for testing that case
# print(index, trend_status, box_low, box_high)

In [56]:
pnf_data_ext = pd.concat([pnf_data,
                         pnf_data[['trend_status', 'range_low', 'range_high']].shift(1)], axis=1)
pnf_data_ext

,High,Low,Close,trend_status,range_low,range_high,trend_status,range_low,range_high
0,38.25,35.75,37.50,1.0,36.0,38.0,NaN,NaN,NaN
1,40.12,39.25,40.00,0.0,0.0,0.0,1.0,36.0,38.0
2,38.50,37.00,37.25,0.0,0.0,0.0,0.0,0.0,0.0
3,37.50,36.50,37.00,0.0,0.0,0.0,0.0,0.0,0.0
4,40.25,37.00,39.00,0.0,0.0,0.0,0.0,0.0,0.0


In [47]:
# alt version
# Process the remaining lines in price_data:
pnf_data[row_count:].apply(lambda row:
                          
                           
                           ,
                          axis=1)

# status can still be 0! create an example for testing that case
# print(index, trend_status, box_low, box_high)

High            40.12
Low             39.25
Close           40.00
trend_status     0.00
range_low        0.00
range_high       0.00
Name: 1, dtype: float64
High            38.50
Low             37.00
Close           37.25
trend_status     0.00
range_low        0.00
range_high       0.00
Name: 2, dtype: float64
High            37.5
Low             36.5
Close           37.0
trend_status     0.0
range_low        0.0
range_high       0.0
Name: 3, dtype: float64
High            40.25
Low             37.00
Close           39.00
trend_status     0.00
range_low        0.00
range_high       0.00
Name: 4, dtype: float64


1    None
2    None
3    None
4    None
dtype: object

In [ ]:
print(trend_status, box_low, box_high)

## Printing the PnF chart

In [ ]:
trend_status = trend_status
box_low = box_low
box_low = box_low

pnf_data = pd.DataFrame({'trend_status': trend_status,
                         'range_low': box_low,
                         'range_high': box_high
                        })
pnf_data

We plot a column each time the **change** bool array is True (note: it's shifted up), then for the last row

In [ ]:
changes = (np.diff(np.sign(trend_status)) != 0)
# We make sure that the a column is generated for the last price line:
changes = np.append(changes, [True])
changes

In [ ]:
# Note that the change column is 'shifted': it's True when a status change is detected on the next price line:
pnf_data['change'] = changes
pnf_data

In [ ]:
ranges = []
trends = []
scale = scale

# should we use .apply() here?
for row in pnf_data[pnf_data['change']].iterrows():
    row = row[1]
    col_range = generate_column_range(scale, row['range_low'], row['range_high'])
    ranges.append(col_range)
    trends.append(row['trend_status'])

columns = list(zip(trends, ranges))

print(columns[:10])

In [ ]:
print(pnf_text(scale, columns))